### 그룹 연산
- 복잡한 데이터를 어떤 기준에 따라 여러 그룹으로 나누어서 관찰할 수 있으며 이런 방식으로 분할 처리하는 것을 그룹 연산이라 함
- 그룹 연산은 데이터를 집계, 변환, 필터링하는데 효율적이며 판다스 groupby() 메서드를 사용함 
- groupby()함수 : DataFrame객체.groupby(기준이 되는 열이름)
- 그룹 객체 만들기(분할)
- 그룹 연산 메소드(적용, 결합)

#### groupby
- 특정 열을 기준으로 데이터셋을 묶는다
- ~~ 별 집계를 할 때 사용한다<br><br>

#### 구문
   - df.groupby('그룹으로묶을기준컬럼')['집계할 컬럼'].집계함수()

In [32]:
import pandas as pd
import seaborn as sns

titanic = sns.load_dataset('titanic')
df=titanic.loc[:,['age','sex','class','fare','survived']] # 행 없이 열에 칼럼명 넣기 


print('승객 수 :', len(df))
print(df.head())

승객 수 : 891
    age     sex  class     fare  survived
0  22.0    male  Third   7.2500         0
1  38.0  female  First  71.2833         1
2  26.0  female  Third   7.9250         1
3  35.0  female  First  53.1000         1
4  35.0    male  Third   8.0500         0


In [33]:
# class열에는 frist, second, third라는 3개의 값들이 들어 있다.
# 이 열을 기준으로 그룹객체를 생성 --> 그렇다면 그룹 객체를 생성하는 기준은 뭘까? (공부)

df.value_counts('class')

class
Third     491
First     216
Second    184
dtype: int64

In [34]:
# class를 그룹 객체로 뽑아내는 이유, 자격여부는? (알아보기) 일단 calss로 
grouped = df.groupby(['class'])

# list(grouped)
# 그룹객체는 반복문을 이용할 수 있음

for key, group in grouped: # 그룹이름 (key)과 그룹으로 쪼개진 데이터(group)를 튜플형태로 묶어서 가지고 있음
    print('* key:',key)
    print('* count:', len(group)) # group이 value값으로 생각하면 됨 (반복문 key, value)
    print(group.head()) 
    print('\n')

* key: First
* count: 216
     age     sex  class     fare  survived
1   38.0  female  First  71.2833         1
3   35.0  female  First  53.1000         1
6   54.0    male  First  51.8625         0
11  58.0  female  First  26.5500         1
23  28.0    male  First  35.5000         1


* key: Second
* count: 184
     age     sex   class     fare  survived
9   14.0  female  Second  30.0708         1
15  55.0  female  Second  16.0000         1
17   NaN    male  Second  13.0000         1
20  35.0    male  Second  26.0000         0
21  34.0    male  Second  13.0000         1


* key: Third
* count: 491
    age     sex  class     fare  survived
0  22.0    male  Third   7.2500         0
2  26.0  female  Third   7.9250         1
4  35.0    male  Third   8.0500         0
5   NaN    male  Third   8.4583         0
7   2.0    male  Third  21.0750         0




In [35]:
# 그룹객체에 대한 일반 연산 --> 개별 그룹별(age, fare, survived 데이터) 연산 수행
# 그룹별 평균 : group객체.mean()

average=grouped.mean()
average # 결과를 보면 연산이 가능한 (숫자형)열에 대해서만 결과를 반환해줌

,age,fare,survived
class,,,
First,38.233441,84.154687,0.629630
Second,29.877630,20.662183,0.472826
Third,25.140620,13.675550,0.242363


In [37]:
# 그룹 추출 : group객체.get_group('그룹이름')
# get_group() 함수를 그룹객체에 적용해 나눠진 그룹중 가져오고 싶은 그룹만 가져옴

# First그룹만 추출해보기
group1 = grouped.get_group('First')
group1.head()


,age,sex,class,fare,survived
1,38.0,female,First,71.2833,1
3,35.0,female,First,53.1000,1
6,54.0,male,First,51.8625,0
11,58.0,female,First,26.5500,1
23,28.0,male,First,35.5000,1


In [38]:
# 여러 열을 기준으로 그룹객체 생성
# DataFrame객체.groupby(기준 열 리스트) 형태로 사용

# class기준으로 먼저 그룹> 그 안에서 sex로 그룹화 > why? 리스트는 순서가 있는 자료형이기 때문

group2 = df.groupby(['class','sex'])

for key, group in group2:
    print('* key', key) # key 값을 보면 한개의 그룹이 아닌 두개의 요소를 가진 튜플임을 확인 할 수 있음=> 멀티 인덱스
    print('* count', len(group))
    print(group.head())
    print('\n')
    

* key ('First', 'female')
* count 94
     age     sex  class      fare  survived
1   38.0  female  First   71.2833         1
3   35.0  female  First   53.1000         1
11  58.0  female  First   26.5500         1
31   NaN  female  First  146.5208         1
52  49.0  female  First   76.7292         1


* key ('First', 'male')
* count 122
     age   sex  class      fare  survived
6   54.0  male  First   51.8625         0
23  28.0  male  First   35.5000         1
27  19.0  male  First  263.0000         0
30  40.0  male  First   27.7208         0
34  28.0  male  First   82.1708         0


* key ('Second', 'female')
* count 76
     age     sex   class     fare  survived
9   14.0  female  Second  30.0708         1
15  55.0  female  Second  16.0000         1
41  27.0  female  Second  21.0000         0
43   3.0  female  Second  41.5792         1
53  29.0  female  Second  26.0000         1


* key ('Second', 'male')
* count 108
     age   sex   class  fare  survived
17   NaN  male  Second  13.

In [39]:
# 이 상태에서 다시 집계함수 적용
# 등석(class)별, 성(sex)별 평균 연령대와 요금, 생존률을 확인할 수 있음
group2.mean()

age        fare  survived
class  sex                                    
First  female  34.611765  106.125798  0.968085
       male    41.281386   67.226127  0.368852
Second female  28.722973   21.970121  0.921053
       male    30.740707   19.741782  0.157407
Third  female  21.750000   16.118810  0.500000
       male    26.507589   12.661633  0.135447

In [ ]:
# 여러가지 그룹연산 메소드
# 판다스에 내장된 기본 집계 함수
# mean(), max(), min(), sum(), count(), size(), var(), std(), describe(),..등

In [40]:
# 다시 타이타닉 데이터 그룹핑
group3 = df.groupby('class')

In [41]:
# 표준편차
group3.std()

,age,fare,survived
class,,,
First,14.802856,78.380373,0.484026
Second,14.001077,13.417399,0.500623
Third,12.495398,11.778142,0.428949


In [42]:
# 최대값
group3.max()

,age,sex,fare,survived
class,,,,
First,80.0,male,512.3292,1
Second,70.0,male,73.5000,1
Third,74.0,male,69.5500,1


In [43]:
# Third'그룹만을 선택해서 group3 이름으로 저장하고 통계요약표를 출력
group4 = grouped.get_group('Third') # 그룹 추출 .get_group('칼럼명')
group4.describe() # 통계요약표 : 그룹명.describe()

,age,fare,survived
count,355.000000,491.000000,491.000000
mean,25.140620,13.675550,0.242363
std,12.495398,11.778142,0.428949
min,0.420000,0.000000,0.000000
25%,18.000000,7.750000,0.000000
50%,24.000000,8.050000,0.000000
75%,32.000000,15.500000,0.000000
max,74.000000,69.550000,1.000000


In [30]:
# 활용도 높은 고급 그룹 연산
 # 여러개의 함수를 여러 열에 적용 : agg()함수
 # 사용자 정의함수 적용 

def min_max(x):
    return x.max()-x.min()

In [44]:
# 일종의 매핑함수를 적용하는 것과 비슷
group3.agg(min_max)

C:\Users\HOME\anaconda3\envs\cakd5\lib\site-packages\pandas\core\groupby\generic.py:303: FutureWarning: Dropping invalid columns in SeriesGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the aggregating function.
  results[key] = self.aggregate(func)


,age,fare,survived
class,,,
First,79.08,512.3292,1
Second,69.33,73.5000,1
Third,73.58,69.5500,1


In [ ]:
# agg함수 여러개의 열에 여러가지 함수를 적용할 수 있음
# 모든열에 여러 함수를 매핑 : group객체.agg([함수1,함수2,함수3,…])
# 각 열마다 다른 함수를 매핑 : group객체.agg({‘열1’: 함수1, ‘열2’:함수2, …})

In [46]:
print(group3.agg(['min','max']))
print('\n')
print(grouped.agg({'fare':['min','max'],'age':'mean'}))

         age           sex       fare           survived    
         min   max     min   max  min       max      min max
class                                                       
First   0.92  80.0  female  male  0.0  512.3292        0   1
Second  0.67  70.0  female  male  0.0   73.5000        0   1
Third   0.42  74.0  female  male  0.0   69.5500        0   1


       fare                  age
        min       max       mean
class                           
First   0.0  512.3292  38.233441
Second  0.0   73.5000  29.877630
Third   0.0   69.5500  25.140620


In [ ]:
# 연산 후 기존 데이터프레임의 형태로 : transform()
# transform()함수는 그룹별로 매핑함를 적용하긴 하지만, 
# 그룹별로 집계하지 않고 원래 데이터프레임의 형태로 반환

# age열을 정규화하는 일반적 방법과 trnasform()을 활용한 방법

In [48]:
# age열을 정규화하는 일반적 방법
age_mean = grouped.age.mean()
age_std = grouped.age.std()

for key, group in grouped.age:
    group_zscore = (group-age_mean.loc[key]) / age_std.loc[key]
    print('* origin:', key)
    print(group_zscore.head(3))
    print('\n')

* origin: First
1   -0.015770
3   -0.218434
6    1.065103
Name: age, dtype: float64


* origin: Second
9    -1.134029
15    1.794317
17         NaN
Name: age, dtype: float64


* origin: Third
0   -0.251342
2    0.068776
4    0.789041
Name: age, dtype: float64




In [50]:
# trnasform()을 활용한 방법

def z_score(x):
    return(x-x.mean())/x.std()

grouped.age.transform(z_score).head(10)
# 그룹을 기준으로 연산을 하고 그 결과를 다시 데이터프레임 형태로 돌려줌

0   -0.251342
1   -0.015770
2    0.068776
3   -0.218434
4    0.789041
5         NaN
6    1.065103
7   -1.851931
8    0.148805
9   -1.134029
Name: age, dtype: float64

### 그룹객체 필터링 : filter()
  - 그룹객체에 대한 filter() 함수는 내장함수의 filter 함수와 전혀 다름
  - group객체.filter(조건식 함수)로 사용하며, 개별원소에 대한 필터링이 아닌  group객체를 필터링 함 즉, 그룹핑한 그룹들중에 가져올 그룹과 안가져올 그룹을 필터링
  - 그룹객체에 매핑함수를 적용할때는 lambda x에 해당하는 x가 각각의 그룹들임

In [52]:
# 그룹별 row수가 200개 이상인 그룹만 추출
grouped = df.groupby('class')
test = grouped.filter(lambda x : len(x) >= 200)

print(test.head())
print('\n')
print(test['class'].unique())
# second그룹은 필터링되어 추출되지 않음

    age     sex  class     fare  survived
0  22.0    male  Third   7.2500         0
1  38.0  female  First  71.2833         1
2  26.0  female  Third   7.9250         1
3  35.0  female  First  53.1000         1
4  35.0    male  Third   8.0500         0


['Third', 'First']
Categories (3, object): ['First', 'Second', 'Third']


In [55]:
# 각 그룹의 age열의 평균이 30미만인 그룹 추출
test = grouped.filter(lambda x : x.age.mean()<30)
print(test.head())
print('\n')
print(test['class'].unique())

    age     sex  class     fare  survived
0  22.0    male  Third   7.2500         0
2  26.0  female  Third   7.9250         1
4  35.0    male  Third   8.0500         0
5   NaN    male  Third   8.4583         0
7   2.0    male  Third  21.0750         0


['Third', 'Second']
Categories (3, object): ['First', 'Second', 'Third']


In [56]:
# 그룹객체에 함수 매핑 : apply()
# group객체.apply(매핑함수) -> 개별 원소가 아닌 그룹별 매핑이 기준

# 각 그룹에 대한 기초통계 정보 확인
grouped.apply(lambda x : x.describe())

age        fare    survived
class                                           
First  count  186.000000  216.000000  216.000000
       mean    38.233441   84.154687    0.629630
       std     14.802856   78.380373    0.484026
       min      0.920000    0.000000    0.000000
       25%     27.000000   30.923950    0.000000
       50%     37.000000   60.287500    1.000000
       75%     49.000000   93.500000    1.000000
       max     80.000000  512.329200    1.000000
Second count  173.000000  184.000000  184.000000
       mean    29.877630   20.662183    0.472826
       std     14.001077   13.417399    0.500623
       min      0.670000    0.000000    0.000000
       25%     23.000000   13.000000    0.000000
       50%     29.000000   14.250000    0.000000
       75%     36.000000   26.000000    1.000000
       max     70.000000   73.500000    1.000000
Third  count  355.000000  491.000000  491.000000
       mean    25.140620   13.675550    0.242363
       std     12.495398   11.778142    0.428949
       min      0.420000    0.000000    0.000000
       25%     18.000000    7.750000    0.000000
       50%     24.000000    8.050000    0.000000
       75%     32.000000   15.500000    0.000000
       max     74.000000   69.550000    1.000000

In [57]:
# 응용문제 : 평균나이가 30미만인 그룹들의 데이터를 출력

age_filter = grouped.apply(lambda x: x.age.mean()<30)
print(age_filter)
print('\n')

for i in age_filter.index: # age_filter의 인덱스
    if age_filter[i]:
        print(grouped.get_group(i).head())
        print('\n')

class
First     False
Second     True
Third      True
dtype: bool


     age     sex   class     fare  survived
9   14.0  female  Second  30.0708         1
15  55.0  female  Second  16.0000         1
17   NaN    male  Second  13.0000         1
20  35.0    male  Second  26.0000         0
21  34.0    male  Second  13.0000         1


    age     sex  class     fare  survived
0  22.0    male  Third   7.2500         0
2  26.0  female  Third   7.9250         1
4  35.0    male  Third   8.0500         0
5   NaN    male  Third   8.4583         0
7   2.0    male  Third  21.0750         0




In [29]:
# class 열, sex 열을 기준으로 분할하여 grouped_two에 저장
grouped_two = df.groupby(['class','sex']) # class 3개 # sex 2개 -> 전체 6개 조합
                            # class에는 first, second, third 
                            # sex에는 female, male
for key, group in grouped_two:
    print('* key:', key)
    print('* number:', len(group))
    print(group.head)
    print('\n')

* key: ('First', 'female')
* number: 94
<bound method NDFrame.head of       age     sex  class      fare  survived
1    38.0  female  First   71.2833         1
3    35.0  female  First   53.1000         1
11   58.0  female  First   26.5500         1
31    NaN  female  First  146.5208         1
52   49.0  female  First   76.7292         1
..    ...     ...    ...       ...       ...
856  45.0  female  First  164.8667         1
862  48.0  female  First   25.9292         1
871  47.0  female  First   52.5542         1
879  56.0  female  First   83.1583         1
887  19.0  female  First   30.0000         1

[94 rows x 5 columns]>


* key: ('First', 'male')
* number: 122
<bound method NDFrame.head of       age   sex  class      fare  survived
6    54.0  male  First   51.8625         0
23   28.0  male  First   35.5000         1
27   19.0  male  First  263.0000         0
30   40.0  male  First   27.7208         0
34   28.0  male  First   82.1708         0
..    ...   ...    ...       ...     

In [25]:
average_two = grouped_two.mean()
average_two

age        fare  survived
class  sex                                    
First  female  34.611765  106.125798  0.968085
       male    41.281386   67.226127  0.368852
Second female  28.722973   21.970121  0.921053
       male    30.740707   19.741782  0.157407
Third  female  21.750000   16.118810  0.500000
       male    26.507589   12.661633  0.135447

In [ ]:
#  ('Third', female) 그룹을 선택해서 group3f에 저장하고 처음 5개행을 출력

In [58]:
# 다중그룹일때 (( )) 대괄호 하나 더 추가
group3f = grouped_two.get_group(('Third','female')) 
group3f.head()

,age,sex,class,fare,survived
2,26.0,female,Third,7.9250,1
8,27.0,female,Third,11.1333,1
10,4.0,female,Third,16.7000,1
14,14.0,female,Third,7.8542,0
18,31.0,female,Third,18.0000,0


In [12]:
group3f[['age','fare']].describe()

,age,fare
count,102.000000,144.000000
mean,21.750000,16.118810
std,12.729964,11.690314
min,0.750000,6.750000
25%,14.125000,7.854200
50%,21.500000,12.475000
75%,29.750000,20.221875
max,63.000000,69.550000


### 적용 - 결합 데이터 집계(agg)
- 집계 연산을 처리하는 사용자 정의함수를 그룹 객체에 적용하려면 agg()메소드 사용
- 모든 열에 여러 함수를 매핑:group객체.agg([함수1,함수2,함수3...])
- 각 열마다 다른 함수를 매핑:group객체.agg({'열1':함수1, '열2':함수2...})

### 변환(transform)
- 원소의 본래 행 인덱스와 열 이름을 기준으로 연산 결과를 반환
- 데이터 변환 연산 : group객체.transform(매핑함수)

### 필터링(filter)
- 그룹 객체에 filter()메소드를 적용할 때 조건식을 가진 함수를 전달하면 조건이 참인 그룹만을 남김
- 그룹 객체 필터링:group객체.filter(조건 함수식)

### 객체에 함수 매핑(apply)
- 판다스 객체의 개별 원소를 특정 함수에 일대일로 매핑. 사용자가 원하는 대부분의 연산을 그룹 객체에 적용
- group객체.apply(매핑함수)

In [ ]:
# (최대값 - 최소값)을 반환하는 사용자 함수를 정의하고 agg() 함수에 인수로 전달하여
# 그룹별로 집계하여 agg_minmax 이름으로 저장 후 처음 5개행을 출력

In [60]:
titanic = sns.load_dataset('titanic')
df = titanic.loc[:,['age','sex','class','fare','survived']]

grouped = df.groupby(['class']) # 클래스별로 그룹화함
grouped.head(1)

,age,sex,class,fare,survived
0,22.0,male,Third,7.2500,0
1,38.0,female,First,71.2833,1
9,14.0,female,Second,30.0708,1


In [61]:
# grouped의 모든 열에 min, max 함수 적용 후 출력

# 클래스별 age, sex, fare, survived 에 따른 min , max 값 구하기
agg_all = grouped.agg(['min','max'])
agg_all.head()

age           sex       fare           survived    
         min   max     min   max  min       max      min max
class                                                       
First   0.92  80.0  female  male  0.0  512.3292        0   1
Second  0.67  70.0  female  male  0.0   73.5000        0   1
Third   0.42  74.0  female  male  0.0   69.5500        0   1

In [69]:
# grouped에서 age열은 mean, fare열은 min, max 함수를 적용하여 출력

# class 별로 age는 mean적용 fare은 min, max 값 적용
agg_sep = grouped.agg({'age':'mean', 'fare':['min','max']}) # 딕셔너리형태로 적용
agg_sep

age fare          
             mean  min       max
class                           
First   38.233441  0.0  512.3292
Second  29.877630  0.0   73.5000
Third   25.140620  0.0   69.5500

In [65]:
# 데이터의 개수가 200개 이상인 그룹만을 필터링하여 데이터프레임으로 변환

grouped_filter = grouped.filter(lambda x : len(x)>=200)
print(grouped_filter,'\n')
grouped_filter.value_counts('class')

      age     sex  class     fare  survived
0    22.0    male  Third   7.2500         0
1    38.0  female  First  71.2833         1
2    26.0  female  Third   7.9250         1
3    35.0  female  First  53.1000         1
4    35.0    male  Third   8.0500         0
..    ...     ...    ...      ...       ...
885  39.0  female  Third  29.1250         0
887  19.0  female  First  30.0000         1
888   NaN  female  Third  23.4500         0
889  26.0    male  First  30.0000         1
890  32.0    male  Third   7.7500         0

[707 rows x 5 columns] 



class
Third     491
First     216
Second      0
dtype: int64

In [66]:
# age열의 평균이 30보다 작은 그룹만 필터링하여 데이터프레임으로 반환

age_filter = grouped.filter(lambda x : x.age.mean()<30)
age_filter.tail(5)

,age,sex,class,fare,survived
884,25.0,male,Third,7.050,0
885,39.0,female,Third,29.125,0
886,27.0,male,Second,13.000,0
888,NaN,female,Third,23.450,0
890,32.0,male,Third,7.750,0


In [67]:
# 각 그룹별 통계요약표 집계

for key, group in grouped:
    print(key, group.describe(),'\n')

First               age        fare    survived
count  186.000000  216.000000  216.000000
mean    38.233441   84.154687    0.629630
std     14.802856   78.380373    0.484026
min      0.920000    0.000000    0.000000
25%     27.000000   30.923950    0.000000
50%     37.000000   60.287500    1.000000
75%     49.000000   93.500000    1.000000
max     80.000000  512.329200    1.000000 

Second               age        fare    survived
count  173.000000  184.000000  184.000000
mean    29.877630   20.662183    0.472826
std     14.001077   13.417399    0.500623
min      0.670000    0.000000    0.000000
25%     23.000000   13.000000    0.000000
50%     29.000000   14.250000    0.000000
75%     36.000000   26.000000    1.000000
max     70.000000   73.500000    1.000000 

Third               age        fare    survived
count  355.000000  491.000000  491.000000
mean    25.140620   13.675550    0.242363
std     12.495398   11.778142    0.428949
min      0.420000    0.000000    0.000000
25%     18.

In [68]:
agg_grouped = grouped.apply(lambda x : x.describe())
agg_grouped

age        fare    survived
class                                           
First  count  186.000000  216.000000  216.000000
       mean    38.233441   84.154687    0.629630
       std     14.802856   78.380373    0.484026
       min      0.920000    0.000000    0.000000
       25%     27.000000   30.923950    0.000000
       50%     37.000000   60.287500    1.000000
       75%     49.000000   93.500000    1.000000
       max     80.000000  512.329200    1.000000
Second count  173.000000  184.000000  184.000000
       mean    29.877630   20.662183    0.472826
       std     14.001077   13.417399    0.500623
       min      0.670000    0.000000    0.000000
       25%     23.000000   13.000000    0.000000
       50%     29.000000   14.250000    0.000000
       75%     36.000000   26.000000    1.000000
       max     70.000000   73.500000    1.000000
Third  count  355.000000  491.000000  491.000000
       mean    25.140620   13.675550    0.242363
       std     12.495398   11.778142    0.428949
       min      0.420000    0.000000    0.000000
       25%     18.000000    7.750000    0.000000
       50%     24.000000    8.050000    0.000000
       75%     32.000000   15.500000    0.000000
       max     74.000000   69.550000    1.000000

In [70]:
# 평균값에서 표준편차의 몇배가 떨어져 있는지를 평가하는 사용자 함수 

def z_score(x):
    return (x-x.mean()) / x.std()

In [71]:
# 위에서 구한 사용자 함수를 이용해 age 컬럼을 transform() 함수로 변환

grouped.age.transform(z_score)[:3] # 평균은 0이고 표준편차는 1 -> 샘플수가 많아질수록 이 값에 근사함

0   -0.251342
1   -0.015770
2    0.068776
Name: age, dtype: float64

In [72]:
# 위에서 구한 사용자 함수를 이용해 age 컬럼을 apply() 함수로 매핑하여 출력

grouped[['age']].apply(lambda x : z_score(x))[:3]

,age
0,-0.251342
1,-0.015770
2,0.068776


In [73]:
df1 = df.groupby(['class','sex'])
type(df1)

pandas.core.groupby.generic.DataFrameGroupBy

In [77]:
# class 값이 First인 행을 선택하여 출력

df2 = df1.mean()
print(df2,'\n')
#type(df2)
df2.loc['First']

                     age        fare  survived
class  sex                                    
First  female  34.611765  106.125798  0.968085
       male    41.281386   67.226127  0.368852
Second female  28.722973   21.970121  0.921053
       male    30.740707   19.741782  0.157407
Third  female  21.750000   16.118810  0.500000
       male    26.507589   12.661633  0.135447 



,age,fare,survived
sex,,,
female,34.611765,106.125798,0.968085
male,41.281386,67.226127,0.368852


In [79]:
# class 값이 First이고 sex값이 female인 행을 선택하여 출력

df2.loc[('First','female')]

age          34.611765
fare        106.125798
survived      0.968085
Name: (First, female), dtype: float64